# iTunes file exchange
This is a script for replacing music files in your iTunes library with new files e.g. higher bitrate without changing ID3 tags

Please update the directories according to your system:

In [48]:
new_files_dir = "/Users/johannkaspar/Downloads/replace_tracks"
itunes_library_xml = '/Users/johannkaspar/Desktop/Mediathek_2022_08_13_13_00.xml'

In [1]:
import urllib.parse
import eyed3
import xml.etree.ElementTree as ET
import os
import shutil
import pandas as pd

In [59]:
pd.set_option('display.max_rows', 100)
pd.options.display.max_colwidth = 150

In [4]:
def xml_to_dict(elements):
    elements_dict = dict()
    for i in range(len(elements) // 2):
        elements_dict[elements[i*2].text] = elements[i*2+1].text
    return elements_dict

In [70]:
# This block creates a DataFrame of all new files meant to replace older ones and their location

file_df = pd.DataFrame(columns=["title", "album_artist", "location"])

for file in os.listdir(new_files_dir):
    if file.endswith(".mp3"):
        location = new_files_dir + "/" + file
        audiofile = eyed3.load(location)
        file_df.loc[len(file_df)] = [audiofile.tag.title, audiofile.tag.album_artist, location]


Invalid date: 0312
Invalid date: 0207
Invalid date: 0308


In [71]:
# This block scans the iTunes XML for tracks with a matching title as in the file_df,
# then checks if artists overlap, if so it copies the ID3 tags from old to new and replaces old by new


root = ET.parse(itunes_library_xml).getroot()

for i, element in enumerate(root.findall('dict/dict/dict')):
    elements_dict = xml_to_dict(element)

    title = elements_dict["Name"]
    itunes_location = urllib.parse.unquote(elements_dict["Location"]).split("file://")[1]


    if title in file_df["title"].to_list():

        file_info = file_df.loc[file_df.title == title]
        downloads_location = file_info.location.values[0]
            
        audiofile_old = eyed3.load(itunes_location)
        audiofile_new = eyed3.load(downloads_location)

        artist_list = urllib.parse.unquote(audiofile_new.tag.artist).split("\x00")
        for _artist in artist_list:
            if _artist in audiofile_old.tag.artist:
                audiofile_new.tag = audiofile_old.tag
                audiofile_new.tag.save()
                print(f"moving f{downloads_location} to {itunes_location}")
                shutil.move(downloads_location, itunes_location)
                file_df = file_df.drop(file_info.index[0])
                break

Invalid date: 0308
Invalid date: 0308
Invalid date: 0207
Invalid date: 0207


moving f/Users/johannkaspar/Downloads/replace_tracks/Bicep - Glue.mp3 to /Volumes/SSD/iTunes/iTunes Media/Music/Bicep/Bicep/02 Glue.mp3
moving f/Users/johannkaspar/Downloads/replace_tracks/x club. - Scum 3 -03.mp3 to /Volumes/SSD/iTunes/iTunes Media/Music/X CLUB_/Scum/01 Scum 3 -03.mp3


Invalid date: 0312
Invalid date: 0312


moving f/Users/johannkaspar/Downloads/replace_tracks/Tic Tac Toe - Isch liebe disch.mp3 to /Volumes/SSD/iTunes/iTunes Media/Music/Tic Tac Toe/Ist der Ruf erst ruiniert .._/1-11 Isch liebe disch.mp3
moving f/Users/johannkaspar/Downloads/replace_tracks/МЕХАНИЧЕСКИЙ ПЁС - Greta Speech 0.1.mp3 to /Volumes/SSD/iTunes/iTunes Media/Music/МЕХАНИЧЕСКИЙ ПЁС/GRETA/1-02 Greta Speech 0.1.mp3


The DataFrame below shows all files that haven't been copied. These ones have to be curated manually.

In [72]:
file_df

,title,album_artist,location
1,Stage One (Space Manoeuvre's Separation Mix),Space Manoeuvres,/Users/johannkaspar/Downloads/replace_tracks/Space Manoeuvres - Stage One (Space Manoeuvre's Separation Mix).mp3


In [73]:
itunes_location = "/Volumes/SSD/iTunes/iTunes Media/Music/Space Manoeuvres/Oid/1-02 Stage One (Seperation Mix).mp3"
downloads_location = "/Users/johannkaspar/Downloads/replace_tracks/Space Manoeuvres - Stage One (Space Manoeuvre's Separation Mix).mp3"

audiofile_old = eyed3.load(itunes_location)
audiofile_new = eyed3.load(downloads_location)

audiofile_new.tag = audiofile_old.tag
audiofile_new.tag.save()
    
shutil.move(downloads_location, itunes_location)

'/Volumes/SSD/iTunes/iTunes Media/Music/Space Manoeuvres/Oid/1-02 Stage One (Seperation Mix).mp3'